In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from random import randint
from tensorflow import keras

In [2]:
X = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test_data  = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
Y          = X["label"]
#print (train_data.shape)
X = X.drop("label", axis=1)

print (X.shape)
print (test_data.shape)

(42000, 784)
(28000, 784)


In [3]:
cuttingNumber = 37000

X         = np.array(X)
Y         = np.array(Y)
test_data = np.array(test_data)

train_data = X[:cuttingNumber]
Y_train    = Y[:cuttingNumber]
val_data   = X[cuttingNumber:]
Y_val      = Y[cuttingNumber:]

train_data = train_data.reshape((-1,28,28,1))
val_data   = val_data.reshape((-1,28,28,1))
test_data  = test_data.reshape((-1,28,28,1))

from keras.utils import to_categorical
Y_train = to_categorical(Y_train)
Y_val   = to_categorical(Y_val)

Using TensorFlow backend.


In [4]:
print (train_data.shape)
print (val_data.shape)
print (Y_train.shape)
print (Y_val.shape)
print (test_data.shape)

(37000, 28, 28, 1)
(5000, 28, 28, 1)
(37000, 10)
(5000, 10)
(28000, 28, 28, 1)


In [5]:
"""
randomIndexes = [randint(0,5000) for i in range(3)]
fig, axs = plt.subplots(3,3,sharex="all", sharey="all")
axs[0][0].set_xticks([])
axs[0][0].set_yticks([])
for i, j in enumerate(randomIndexes):
    if (i==0):
        axs[i][0].title.set_text("Training Data")
        axs[i][1].title.set_text("Validation Data")
        axs[i][2].title.set_text("Test Data")
    axs[i][0].imshow(train_data[j], cmap="binary")
    axs[i][1].imshow(val_data[j], cmap="binary")
    axs[i][2].imshow(test_data[j], cmap="binary")
"""

'\nrandomIndexes = [randint(0,5000) for i in range(3)]\nfig, axs = plt.subplots(3,3,sharex="all", sharey="all")\naxs[0][0].set_xticks([])\naxs[0][0].set_yticks([])\nfor i, j in enumerate(randomIndexes):\n    if (i==0):\n        axs[i][0].title.set_text("Training Data")\n        axs[i][1].title.set_text("Validation Data")\n        axs[i][2].title.set_text("Test Data")\n    axs[i][0].imshow(train_data[j], cmap="binary")\n    axs[i][1].imshow(val_data[j], cmap="binary")\n    axs[i][2].imshow(test_data[j], cmap="binary")\n'

In [6]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(input_shape=(28,28,1),filters = 8, kernel_size=(2,2), padding="same"))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(filters = 16, kernel_size=(2,2),padding="same"))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=256,activation = "relu"))
model.add(keras.layers.Dense(units=64, activation = "relu"))
model.add(keras.layers.Dense(units=10, activation = "softmax"))
model.compile(optimizer="Adam", loss = "categorical_crossentropy", metrics = ["accuracy"])
model.fit (train_data,Y_train, epochs=15, validation_data = (val_data,Y_val))
model.evaluate(val_data, Y_val)

Train on 37000 samples, validate on 5000 samples
Epoch 1/15
37000/37000 [==============================] - 9s 233us/sample - loss: 0.5323 - accuracy: 0.9045 - val_loss: 0.1533 - val_accuracy: 0.9528
Epoch 2/15
37000/37000 [==============================] - 4s 118us/sample - loss: 0.1037 - accuracy: 0.9686 - val_loss: 0.1135 - val_accuracy: 0.9670
Epoch 3/15
37000/37000 [==============================] - 5s 123us/sample - loss: 0.0722 - accuracy: 0.9776 - val_loss: 0.1255 - val_accuracy: 0.9638
Epoch 4/15
37000/37000 [==============================] - 4s 121us/sample - loss: 0.0596 - accuracy: 0.9810 - val_loss: 0.1038 - val_accuracy: 0.9700
Epoch 5/15
37000/37000 [==============================] - 5s 136us/sample - loss: 0.0542 - accuracy: 0.9837 - val_loss: 0.0808 - val_accuracy: 0.9766
Epoch 6/15
37000/37000 [==============================] - 5s 139us/sample - loss: 0.0382 - accuracy: 0.9883 - val_loss: 0.0983 - val_accuracy: 0.9752
Epoch 7/15
37000/37000 [===========================

[0.13059550819417892, 0.975]

In [7]:
predictions = model.predict(test_data)
predictions = np.argmax(predictions, axis=1)
IdRange     = np.arange(1, predictions.shape[0]+1)
output = pd.DataFrame({'ImageId':IdRange,'Label':predictions})
output.to_csv("my_submission.csv", index=False)
print ("Your submission was succesfully saved!")

Your submission was succesfully saved!
